# Key Levels Detection (Close)

In [9]:
import warnings
warnings.filterwarnings('ignore')

import time
from datetime import date, datetime
from datetime import timedelta
from dateutil.relativedelta import relativedelta

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import pandas_ta as ta
from ps_candlestick import get_type_candlestick
from ps_candlestick import has_reversal_pattern

In [10]:
import os
from pathlib import Path
notebook_path = os.getcwd()
algo_dir = Path(notebook_path).parent.parent
csv_file = str(algo_dir) + '/vn-stock-data/VN30ps/VN30F1M_5minutes.csv'
is_file = os.path.isfile(csv_file)
if is_file:
    dataset = pd.read_csv(csv_file, index_col='Date', parse_dates=True)
else:
    dataset = pd.read_csv("https://raw.githubusercontent.com/zuongthaotn/vn-stock-data/main/VN30ps/VN30F1M_5minutes.csv", index_col='Date', parse_dates=True)
dataset

,Open,High,Low,Close,Volume
Date,,,,,
2018-08-13 09:00:00,943.5,943.6,942.9,943.1,1812
2018-08-13 09:05:00,943.1,943.5,942.9,943.3,1323
2018-08-13 09:10:00,943.2,943.3,942.6,943.1,1207
2018-08-13 09:15:00,943.1,943.1,942.3,942.6,1196
2018-08-13 09:20:00,942.6,943.7,942.4,943.7,1765
...,...,...,...,...,...
2024-07-09 14:10:00,1316.0,1321.0,1315.5,1320.8,8273
2024-07-09 14:15:00,1321.3,1321.9,1318.4,1319.1,11670
2024-07-09 14:20:00,1319.0,1319.0,1317.3,1317.5,6217


In [11]:
data = dataset.copy()
data['max_5bars_prev'] = data['Close'].rolling(5).max()
data['max_5bars_next'] = data['Close'].shift(-4).rolling(5).max()
data['min_5bars_prev'] = data['Close'].rolling(5).min()
data['min_5bars_next'] = data['Close'].shift(-4).rolling(5).min()
data

,Open,High,Low,Close,Volume,max_5bars_prev,max_5bars_next,min_5bars_prev,min_5bars_next
Date,,,,,,,,,
2018-08-13 09:00:00,943.5,943.6,942.9,943.1,1812,NaN,NaN,NaN,NaN
2018-08-13 09:05:00,943.1,943.5,942.9,943.3,1323,NaN,NaN,NaN,NaN
2018-08-13 09:10:00,943.2,943.3,942.6,943.1,1207,NaN,NaN,NaN,NaN
2018-08-13 09:15:00,943.1,943.1,942.3,942.6,1196,NaN,NaN,NaN,NaN
2018-08-13 09:20:00,942.6,943.7,942.4,943.7,1765,943.7,945.3,942.6,943.7
...,...,...,...,...,...,...,...,...,...
2024-07-09 14:10:00,1316.0,1321.0,1315.5,1320.8,8273,1320.8,1320.8,1315.8,1317.5
2024-07-09 14:15:00,1321.3,1321.9,1318.4,1319.1,11670,1320.8,NaN,1315.8,NaN
2024-07-09 14:20:00,1319.0,1319.0,1317.3,1317.5,6217,1320.8,NaN,1315.8,NaN


In [12]:
data['is_r_keylevel'] = data.apply(lambda r: True if (r['Close'] == r['max_5bars_prev'] and r['Close'] == r['max_5bars_next']) else False, axis=1)
data['is_s_keylevel'] = data.apply(lambda r: True if (r['Close'] == r['min_5bars_prev'] and r['Close'] == r['min_5bars_next']) else False, axis=1)
data

,Open,High,Low,Close,Volume,max_5bars_prev,max_5bars_next,min_5bars_prev,min_5bars_next,is_r_keylevel,is_s_keylevel
Date,,,,,,,,,,,
2018-08-13 09:00:00,943.5,943.6,942.9,943.1,1812,NaN,NaN,NaN,NaN,False,False
2018-08-13 09:05:00,943.1,943.5,942.9,943.3,1323,NaN,NaN,NaN,NaN,False,False
2018-08-13 09:10:00,943.2,943.3,942.6,943.1,1207,NaN,NaN,NaN,NaN,False,False
2018-08-13 09:15:00,943.1,943.1,942.3,942.6,1196,NaN,NaN,NaN,NaN,False,False
2018-08-13 09:20:00,942.6,943.7,942.4,943.7,1765,943.7,945.3,942.6,943.7,False,False
...,...,...,...,...,...,...,...,...,...,...,...
2024-07-09 14:10:00,1316.0,1321.0,1315.5,1320.8,8273,1320.8,1320.8,1315.8,1317.5,True,False
2024-07-09 14:15:00,1321.3,1321.9,1318.4,1319.1,11670,1320.8,NaN,1315.8,NaN,False,False
2024-07-09 14:20:00,1319.0,1319.0,1317.3,1317.5,6217,1320.8,NaN,1315.8,NaN,False,False


In [15]:
has_keylevel = data[(data['is_r_keylevel'] == True) | (data['is_s_keylevel'] == True)]
has_keylevel['count'] = '1'
has_keylevel['keylevels'] = has_keylevel['Close']
has_keylevel

,Open,High,Low,Close,Volume,max_5bars_prev,max_5bars_next,min_5bars_prev,min_5bars_next,is_r_keylevel,is_s_keylevel,count,keylevels
Date,,,,,,,,,,,,,
2018-08-13 09:25:00,943.8,945.9,943.7,945.3,2469,945.3,945.3,942.6,943.7,True,False,1,945.3
2018-08-13 09:50:00,945.0,946.0,944.3,946.0,1213,946.0,946.0,943.7,945.0,True,False,1,946.0
2018-08-13 09:55:00,946.2,946.4,945.6,946.0,1873,946.0,946.0,943.7,945.0,True,False,1,946.0
2018-08-13 10:50:00,943.6,943.8,942.3,943.2,1541,944.8,946.1,943.2,943.2,False,True,1,943.2
2018-08-13 13:10:00,948.0,948.7,947.3,948.7,1230,948.7,948.7,947.0,947.0,True,False,1,948.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-07-09 10:50:00,1308.1,1309.0,1307.8,1307.9,2471,1309.7,1312.6,1307.9,1307.9,False,True,1,1307.9
2024-07-09 13:05:00,1316.8,1319.4,1316.7,1319.4,10931,1319.4,1319.4,1312.3,1317.9,True,False,1,1319.4
2024-07-09 13:35:00,1316.3,1316.9,1315.7,1316.0,5197,1318.3,1318.3,1316.0,1316.0,False,True,1,1316.0


In [47]:
has_keylevel[(has_keylevel.index > '2024-07-05 00:00:00') & (has_keylevel.index < '2024-07-05 15:00:00')]

,Open,High,Low,Close,Volume,max_5bars_prev,max_5bars_next,min_5bars_prev,min_5bars_next,is_r_keylevel,is_s_keylevel,count,keylevels
Date,,,,,,,,,,,,,
2024-07-05 09:15:00,1309.4,1310.6,1309.4,1310.6,2466,1310.6,1310.6,1308.6,1308.6,True,False,1,1310.6
2024-07-05 09:30:00,1308.7,1309.0,1307.8,1308.6,4492,1310.6,1309.7,1308.6,1308.6,False,True,1,1308.6
2024-07-05 10:25:00,1313.8,1314.4,1313.0,1314.1,3278,1314.1,1314.1,1311.6,1309.8,True,False,1,1314.1
2024-07-05 11:10:00,1309.2,1309.5,1308.7,1308.7,1630,1310.0,1309.3,1308.7,1308.7,False,True,1,1308.7
2024-07-05 11:25:00,1308.8,1309.3,1308.6,1308.7,2265,1309.3,1309.9,1308.7,1308.7,False,True,1,1308.7
2024-07-05 11:30:00,1309.0,1309.0,1308.7,1308.7,72,1309.3,1309.9,1308.7,1308.7,False,True,1,1308.7
2024-07-05 13:35:00,1310.4,1310.8,1309.7,1310.7,2090,1310.7,1310.7,1309.3,1307.8,True,False,1,1310.7
2024-07-05 13:40:00,1310.7,1310.7,1307.8,1307.8,4316,1310.7,1309.5,1307.8,1307.8,False,True,1,1307.8
2024-07-05 14:30:00,1316.1,1316.4,1316.1,1316.4,355,1316.4,1316.4,1311.0,1315.8,True,False,1,1316.4


In [33]:
def cal_keylevels(tick):
    if tick.empty:
        return ''
    keylevels = []
    tick = tick.sort_values(ascending=False)
    i = 0
    while i < len(tick):
        if i == 0 or i == len(tick)-1:
            keylevels.append(tick[i])
        else:
            if tick[i-1] > tick[i] + 2 or tick[i] > tick[i+1] + 2:
                keylevels.append(tick[i])
        i = i+1
    return ", ".join(map(str, keylevels))

has_keylevel_day = has_keylevel.resample("D").agg({
        'count': 'count',
        'keylevels': cal_keylevels
        #'keylevels': lambda x: ", ".join(map(str, x))
    })
has_keylevel_day = has_keylevel_day[has_keylevel_day['count'] != 0]

In [34]:
has_keylevel_day[['count']]

,count
Date,
2018-08-13,7
2018-08-14,7
2018-08-15,10
2018-08-16,10
2018-08-17,7
...,...
2024-07-03,9
2024-07-04,11
2024-07-05,9


In [35]:
has_keylevel_day[['count']].min()

count    2
dtype: int64

In [36]:
has_keylevel_day['count'].max()

21

In [37]:
has_keylevel_day[['keylevels']]

,keylevels
Date,
2018-08-13,"948.7, 945.3, 943.2"
2018-08-14,"961.2, 957.1, 952.3"
2018-08-15,"962.1, 958.2, 955.2, 955.2"
2018-08-16,"944.3, 936.2"
2018-08-17,"954.3, 951.5, 948.8, 946.5"
...,...
2024-07-03,"1308.4, 1305.5, 1297.8, 1294.9"
2024-07-04,"1312.5, 1309.8, 1307.0, 1304.5"
2024-07-05,"1316.4, 1314.1, 1310.7, 1307.8"


In [38]:
has_keylevel_day['keylevels'] = has_keylevel_day['keylevels'].shift(1)
join_data = has_keylevel_day[['keylevels']]

In [39]:
data2 = dataset.copy()
data2['cs'] = data2.apply(lambda r: get_type_candlestick(r), axis=1)
data2['cs_shift_1'] = data2['cs'].shift(1)
data2['cs_shift_2'] = data2['cs'].shift(2)
data2['cs_reversal'] = data2.apply(lambda r: has_reversal_pattern(r), axis=1)
data2["ema_line"] = ta.ema(data2["Close"], length=20)
data2['above_ma'] = data2.apply(lambda r: 1 if r['Close'] > r['ema_line'] else 0, axis=1)
data2['below_ma'] = data2.apply(lambda r: 1 if r['Close'] < r['ema_line'] else 0, axis=1)
data2['total_above_ma'] = data2['above_ma'].rolling(150).sum()
data2['total_below_ma'] = data2['below_ma'].rolling(150).sum()
data2['trend'] = data2.apply(lambda r: 'switch' if r['total_above_ma'] == r['total_below_ma'] else ('up' if r['total_above_ma'] > r['total_below_ma'] else 'down'), axis=1)

In [40]:
data2 = data2.assign(time_d=pd.PeriodIndex(data2.index, freq='1D').to_timestamp())
data2 = pd.merge(data2, join_data, left_on="time_d", right_index=True, how="left")

In [41]:
data2 = data2[data2['keylevels'].notnull()]
data2

,Open,High,Low,Close,Volume,cs,cs_shift_1,cs_shift_2,cs_reversal,ema_line,above_ma,below_ma,total_above_ma,total_below_ma,trend,time_d,keylevels
Date,,,,,,,,,,,,,,,,,
2018-08-14 09:00:00,955.5,955.5,954.7,954.9,1103,white,black,black,0,950.522473,1,0,NaN,NaN,down,2018-08-14,"948.7, 945.3, 943.2"
2018-08-14 09:05:00,954.8,955.0,954.3,955.0,530,black,white,black,0,950.948904,1,0,NaN,NaN,down,2018-08-14,"948.7, 945.3, 943.2"
2018-08-14 09:10:00,955.0,955.1,954.7,955.0,509,doji,black,white,0,951.334723,1,0,NaN,NaN,down,2018-08-14,"948.7, 945.3, 943.2"
2018-08-14 09:15:00,955.0,957.0,955.0,956.9,1758,big_body_white,doji,black,1,951.864749,1,0,NaN,NaN,down,2018-08-14,"948.7, 945.3, 943.2"
2018-08-14 09:20:00,956.5,956.6,955.6,955.8,1230,white,big_body_white,doji,0,952.239535,1,0,NaN,NaN,down,2018-08-14,"948.7, 945.3, 943.2"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-07-09 14:10:00,1316.0,1321.0,1315.5,1320.8,8273,black,doji,white,1,1315.924825,1,0,87.0,63.0,up,2024-07-09,"1316.4, 1315.8, 1312.8, 1311.4, 1309.2, 1306.0"
2024-07-09 14:15:00,1321.3,1321.9,1318.4,1319.1,11670,white,black,doji,0,1316.227223,1,0,87.0,63.0,up,2024-07-09,"1316.4, 1315.8, 1312.8, 1311.4, 1309.2, 1306.0"
2024-07-09 14:20:00,1319.0,1319.0,1317.3,1317.5,6217,big_body_black,white,black,0,1316.348440,1,0,87.0,63.0,up,2024-07-09,"1316.4, 1315.8, 1312.8, 1311.4, 1309.2, 1306.0"


In [42]:
def cross_keylevel(row):
    cross = ''
    keylevels = str(row['keylevels']).split(", ")
    current_price = row['Close']
    for keylevel in keylevels:
        if row['High'] > float(keylevel) > row['Low']:
            cross = 'cross'
    return cross
data2['cross'] = data2.apply(lambda r: cross_keylevel(r), axis=1)

In [43]:
data2

,Open,High,Low,Close,Volume,cs,cs_shift_1,cs_shift_2,cs_reversal,ema_line,above_ma,below_ma,total_above_ma,total_below_ma,trend,time_d,keylevels,cross
Date,,,,,,,,,,,,,,,,,,
2018-08-14 09:00:00,955.5,955.5,954.7,954.9,1103,white,black,black,0,950.522473,1,0,NaN,NaN,down,2018-08-14,"948.7, 945.3, 943.2",
2018-08-14 09:05:00,954.8,955.0,954.3,955.0,530,black,white,black,0,950.948904,1,0,NaN,NaN,down,2018-08-14,"948.7, 945.3, 943.2",
2018-08-14 09:10:00,955.0,955.1,954.7,955.0,509,doji,black,white,0,951.334723,1,0,NaN,NaN,down,2018-08-14,"948.7, 945.3, 943.2",
2018-08-14 09:15:00,955.0,957.0,955.0,956.9,1758,big_body_white,doji,black,1,951.864749,1,0,NaN,NaN,down,2018-08-14,"948.7, 945.3, 943.2",
2018-08-14 09:20:00,956.5,956.6,955.6,955.8,1230,white,big_body_white,doji,0,952.239535,1,0,NaN,NaN,down,2018-08-14,"948.7, 945.3, 943.2",
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-07-09 14:10:00,1316.0,1321.0,1315.5,1320.8,8273,black,doji,white,1,1315.924825,1,0,87.0,63.0,up,2024-07-09,"1316.4, 1315.8, 1312.8, 1311.4, 1309.2, 1306.0",cross
2024-07-09 14:15:00,1321.3,1321.9,1318.4,1319.1,11670,white,black,doji,0,1316.227223,1,0,87.0,63.0,up,2024-07-09,"1316.4, 1315.8, 1312.8, 1311.4, 1309.2, 1306.0",
2024-07-09 14:20:00,1319.0,1319.0,1317.3,1317.5,6217,big_body_black,white,black,0,1316.348440,1,0,87.0,63.0,up,2024-07-09,"1316.4, 1315.8, 1312.8, 1311.4, 1309.2, 1306.0",


In [44]:
kz = data2.iloc[-1]
keylevels = str(kz['keylevels']).split(", ")
keylevels = [float(i) for i in keylevels]
keylevels.sort(reverse=True)
keylevels

[1316.4, 1315.8, 1312.8, 1311.4, 1309.2, 1306.0]

In [46]:
selected_day = data2[(data2.index > '2024-07-09 00:00:00') & (data2.index < '2024-07-09 15:00:00')]
selected_day[selected_day['cross'] == 'cross']

,Open,High,Low,Close,Volume,cs,cs_shift_1,cs_shift_2,cs_reversal,ema_line,above_ma,below_ma,total_above_ma,total_below_ma,trend,time_d,keylevels,cross
Date,,,,,,,,,,,,,,,,,,
2024-07-09 09:00:00,1316.2,1316.9,1314.5,1315.1,5509,white,doji,doji,1,1310.954094,1,0,83.0,67.0,up,2024-07-09,"1316.4, 1315.8, 1312.8, 1311.4, 1309.2, 1306.0",cross
2024-07-09 09:25:00,1313.8,1313.8,1311.5,1311.5,4818,big_body_black,white,white,0,1312.006975,0,1,82.0,68.0,up,2024-07-09,"1316.4, 1315.8, 1312.8, 1311.4, 1309.2, 1306.0",cross
2024-07-09 09:30:00,1311.3,1311.3,1307.4,1308.2,11630,white,big_body_black,white,0,1311.644406,0,1,81.0,69.0,up,2024-07-09,"1316.4, 1315.8, 1312.8, 1311.4, 1309.2, 1306.0",cross
2024-07-09 09:40:00,1307.9,1307.9,1305.9,1306.5,7154,white,spinning_top,white,1,1310.814582,0,1,79.0,71.0,up,2024-07-09,"1316.4, 1315.8, 1312.8, 1311.4, 1309.2, 1306.0",cross
2024-07-09 09:45:00,1306.6,1306.8,1304.7,1304.8,6854,big_body_black,white,spinning_top,0,1310.241765,0,1,78.0,72.0,up,2024-07-09,"1316.4, 1315.8, 1312.8, 1311.4, 1309.2, 1306.0",cross
2024-07-09 09:50:00,1304.6,1306.2,1304.5,1305.8,4147,black,big_body_black,white,0,1309.818740,0,1,77.0,73.0,up,2024-07-09,"1316.4, 1315.8, 1312.8, 1311.4, 1309.2, 1306.0",cross
2024-07-09 09:55:00,1305.7,1306.3,1305.4,1306.2,2744,black,black,big_body_black,0,1309.474098,0,1,76.0,74.0,up,2024-07-09,"1316.4, 1315.8, 1312.8, 1311.4, 1309.2, 1306.0",cross
2024-07-09 10:00:00,1306.3,1306.3,1305.1,1305.9,3134,spinning_top,black,black,0,1309.133707,0,1,75.0,75.0,switch,2024-07-09,"1316.4, 1315.8, 1312.8, 1311.4, 1309.2, 1306.0",cross
2024-07-09 10:30:00,1307.0,1309.3,1307.0,1308.9,5610,black,black,black,0,1308.382672,1,0,74.0,76.0,down,2024-07-09,"1316.4, 1315.8, 1312.8, 1311.4, 1309.2, 1306.0",cross


In [30]:
selected_day.iloc[-1]['keylevels']

'1305.4, 1299.0, 1293.8'

In [31]:
selected_day = data2[(data2.index > '2024-06-27 00:00:00') & (data2.index < '2024-06-27 15:00:00')]
selected_day[selected_day['cross'] == 'cross']

,Open,High,Low,Close,Volume,cs,cs_shift_1,cs_shift_2,cs_reversal,ema_line,above_ma,below_ma,total_above_ma,total_below_ma,trend,time_d,keylevels,cross
Date,,,,,,,,,,,,,,,,,,
2024-06-27 09:10:00,1284.4,1284.9,1282.7,1284.3,4146,spinning_top,spinning_top,doji,1,1284.950877,0,1,35.0,115.0,down,2024-06-27,"1291.5, 1288.3, 1282.8, 1280.7, 1276.4",cross
2024-06-27 09:30:00,1284.4,1284.4,1282.2,1282.8,5752,white,white,doji,0,1284.809807,0,1,36.0,114.0,down,2024-06-27,"1291.5, 1288.3, 1282.8, 1280.7, 1276.4",cross
2024-06-27 10:00:00,1282.8,1284.0,1282.5,1283.9,2975,black,white,spinning_top,0,1284.258696,0,1,36.0,114.0,down,2024-06-27,"1291.5, 1288.3, 1282.8, 1280.7, 1276.4",cross
2024-06-27 10:40:00,1283.8,1284.4,1281.7,1281.9,3928,white,black,spinning_top,0,1284.102224,0,1,40.0,110.0,down,2024-06-27,"1291.5, 1288.3, 1282.8, 1280.7, 1276.4",cross
2024-06-27 10:50:00,1282.1,1282.9,1281.8,1282.9,2159,black,spinning_top,white,1,1283.823816,0,1,40.0,110.0,down,2024-06-27,"1291.5, 1288.3, 1282.8, 1280.7, 1276.4",cross
2024-06-27 10:55:00,1282.6,1283.1,1281.2,1281.4,2738,white,black,spinning_top,0,1283.592976,0,1,40.0,110.0,down,2024-06-27,"1291.5, 1288.3, 1282.8, 1280.7, 1276.4",cross
2024-06-27 11:05:00,1282.5,1284.5,1282.4,1283.8,4313,black,black,white,0,1283.518513,1,0,41.0,109.0,down,2024-06-27,"1291.5, 1288.3, 1282.8, 1280.7, 1276.4",cross
2024-06-27 11:10:00,1283.7,1283.9,1282.5,1282.6,2052,white,black,black,0,1283.431036,0,1,41.0,109.0,down,2024-06-27,"1291.5, 1288.3, 1282.8, 1280.7, 1276.4",cross
2024-06-27 11:15:00,1282.6,1283.7,1282.1,1283.6,2474,black,white,black,0,1283.447128,1,0,42.0,108.0,down,2024-06-27,"1291.5, 1288.3, 1282.8, 1280.7, 1276.4",cross


In [32]:
selected_day.iloc[0]['keylevels']

'1291.5, 1288.3, 1282.8, 1280.7, 1276.4'